## Document Loaders
Load PdF from web

In [5]:
# load environment variables
# openai key is loaded in .env file. load the environment variables
%load_ext dotenv
%dotenv


The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


Import pdf document loader

In [7]:
from langchain_community.document_loaders import PyPDFLoader

# Load the PdF document using langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader("https://www.first.org/resources/guides/Establishing-a-Certification-Authority-CA.pdf")
pages = loader.load_and_split()

print(len(pages))

66


## Embeddings - use HuggingFace embedding
Embeddings can be used to create a numerical representation of textual data. This numerical representation is useful because it can be used to find similar documents

In [8]:
from langchain_community.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


## Vector Database 
Vector databases provide the ability to store and retrieve vectors as high-dimensional points. They add additional capabilities for efficient and fast lookup of nearest-neighbors in the N-dimensional space.

In [9]:
from langchain_community.vectorstores import FAISS
vector = FAISS.from_documents(pages, embeddings)

Retriver 

In [12]:
response = vector.similarity_search("What is a Certification Authority?",k=5)
print(len(response))

retriver = vector.as_retriever(search_type="similarity", search_kwargs={"k": 5})
response = retriver.invoke("What is a Certification Authority?")
print(response)
print(len(response))


5
[Document(page_content='Thailand National Root Certification Authority : Thailand NRCA  \nElectronic Transactio ns Development Agency  Establishing a CA  \n \n- 12 - \n \n1.2 The components of PKI  \nThere are several components involved in PKI, combining s ervices and technologies. All  those r oles can be \nperformed by the same organization, but some of them may be outsourced (with limitations).  \n \nFigure 6: Relationship of the various PKI components  \n1.2.1 Certification Authority (CA)  \nThe Certification Authority, or Certificate Authority (both names are used interchangeably), is the party that \nsigns digital certificates.  \nThere are 2 types of CAs: public and private. Public CAs are those vendors you can find on the Internet \nwhen you are looking to buy a certificate. Private (a.k.a. Internal) CAs are an IT asset located inside \norganizations, to create certificates needed by staff and devices for secure operations.  \n1.2.2 Registration Authority (RA)  \nThe Registr

In [14]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

llm = ChatOpenAI(openai_api_key=getenv("OPENAI_API_KEY"))

template = '''Answer the question based only on the following context:
<context>
{context}
<context>

Question: {question}
'''
prompts = ChatPromptTemplate.from_template(template=template)
# To make it as easy as possible to create custom chains, we’ve implemented a “Runnable” protocol.
# Many LangChain components implement the Runnable protocol, including chat models, LLMs, output parsers, retrievers, prompt templates, and more.
# Runnable to passthrough inputs unchanged or with additional keys.
retriver_chain = (
    { "context": retriver, "question": RunnablePassthrough()}
    | prompts
    | llm 
    | StrOutputParser()
)
 
from langchain_core.messages import HumanMessage
response = retriver_chain.invoke("What is a Certification Authority?")
print("Response:=", response)

response = retriver_chain.invoke("What is a Intermediate Certification Authority?")
print("Response:=", response)


Response:= A Certification Authority is the party that signs digital certificates.
Response:= An Intermediate Certification Authority (CA) is an entity that operates under the trust of a Root CA and can potentially be operated by third parties. It can be used for segmentation, where different use cases or services with different quality levels are managed under different Intermediate CAs.
